In [1]:
import re
import pandas as pd
import numpy as np 
# import matplotlib.pyplot as plt
# import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
from tensorflow.keras.models import load_model
import json

In [11]:
# import tensorflow
# import keras
# import numpy as np
# import pandas as pd
# import h5py
# import konlpy 

# print('tensorflow ',tensorflow.__version__)
# print('keras', keras.__version__)
# print('numpy', np.__version__)
# print('padnas', pd.__version__)
# print('h5py', h5py.__version__)
# print('konlpy', konlpy.__version__)

In [9]:
class psng_predict_all:
    def __init__(self, df):
        self.df = df
        path = 'C:/Users/PiGiraffe0/Documents/triton-modeldata/modeldata'
        self.loaded_model = load_model(f'{path}/new_model_01_v2.h5')
        train_data = pd.read_csv(f'{path}/train_data_main.csv', encoding='euc-kr')
        test_data = pd.read_csv(f'{path}/test_data_main.csv', encoding='euc-kr')
        X_train = train_data['tokenized'].apply(eval).values
        self.X_test = test_data['tokenized'].apply(eval).values
        self.y_test = np.array(test_data['label'].values)
        self.X_test_tkd = None
        vocab_size = 87912 # change
        self.tokenizer = Tokenizer(vocab_size, oov_token='OOV')
        self.tokenizer.fit_on_texts(X_train)
        self.max_len = 250

    def preprocessing(self):
        okt = Okt()
        R_frm = self.df.copy()
        R_frm['리뷰'] = R_frm['리뷰'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
        R_frm['리뷰'].replace('', np.nan, inplace=True)
        R_frm['리뷰'] = R_frm['리뷰'].astype(str)
        stopwords = ['도', '는', '다', '의', '가', '이', '은',
                     '한', '에', '하', '고', '을', '를', '인', '듯',
                     '과', '와', '네', '들', '듯', '지', '임', '게',
                     '는', '이', '했', '슴', '음', '것', '거', '로',
                     '들', '거', '곳', '분', '원', '더', '왜', '해',
                     '수', '할', '그', '함', '돈', '번', '두', '개',
                     '건', '내', '저', '만', '갈', '걸', '제', '명',
                     '분',
                     '인데', '이가', '했었', '해서',
                     '습니다', '했는데', '입니다']

        okt = Okt()
        R_frm['tokenized'] = R_frm['리뷰'].apply(okt.pos)
        # R_frm['tokenized'] = R_frm['리뷰'].apply(lambda x: okt.pos(x, norm=True, stem=True))
        R_frm['tokenized'] = R_frm['tokenized'].apply(
            lambda x: [word for word, shape in x if shape in ['Verb', 'Adjective', 'Noun', 'VerbPrefix'] if word not in stopwords])
        R_pred = R_frm['tokenized'].values
        return R_pred

    def model_test(self):
        tokenizer = self.tokenizer
        X_test = tokenizer.texts_to_sequences(self.X_test)
        y_test = self.y_test
        X_test = pad_sequences(X_test, maxlen=self.max_len)
        print("\n 테스트 정확도: %.4f" % (self.loaded_model.evaluate(X_test, y_test)[1]))

    def word_index(self):
        tokenizer = self.tokenizer
        print(tokenizer.word_index)

    def predict(self):
        R_pred = self.preprocessing()
        max_len = self.max_len
        tokenizer = self.tokenizer
        result = []
        pred = tokenizer.texts_to_sequences(R_pred)
        pred = pad_sequences(pred, maxlen=max_len)
        score = self.loaded_model.predict(pred)
        return score

    def prediction(self):
        result = []
        score = self.predict()
        for num in range(len(score)):
            if score[num][0] > 0.9:
                result.append('0')
            elif score[num][0] < 0.1:
                result.append('1')
            else:
                result.append('2')
        return result


In [3]:
class psng_predict_all_v2:
    def __init__(self, df):
        self.df = df
        path = ''
        self.loaded_model = load_model(f'{path}posneg_no_oneword.h5')
        train_data = pd.read_csv(f'{path}train_data_main_v2.csv', encoding='euc-kr')
        test_data = pd.read_csv(f'{path}test_data_main_v2.csv', encoding='euc-kr')
        X_train = train_data['tokenized'].apply(eval).values
        self.X_test = test_data['tokenized'].apply(eval).values
        self.y_test = np.array(test_data['label'].values)
        self.X_test_tkd = None
        vocab_size = 20894 # change
        self.tokenizer = Tokenizer(vocab_size, oov_token='OOV')
        self.tokenizer.fit_on_texts(X_train)
        self.max_len = 250

    def preprocessing(self):
        okt = Okt()
        R_frm = self.df.copy()
        R_frm['리뷰'] = R_frm['리뷰'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
        R_frm['리뷰'].replace('', np.nan, inplace=True)
        R_frm['리뷰'] = R_frm['리뷰'].astype(str)
        stopwords = ['도', '는', '다', '의', '가', '이', '은',
                     '한', '에', '하', '고', '을', '를', '인', '듯',
                     '과', '와', '네', '들', '듯', '지', '임', '게',
                     '는', '이', '했', '슴', '음', '것', '거', '로',
                     '들', '거', '곳', '분', '원', '더', '왜', '해',
                     '수', '할', '그', '함', '돈', '번', '두', '개',
                     '건', '내', '저', '만', '갈', '걸', '제', '명',
                     '분',
                     '인데', '이가', '했었', '해서',
                     '습니다', '했는데', '입니다']

        okt = Okt()
        # R_frm['tokenized'] = R_frm['리뷰'].apply(okt.pos)
        R_frm['tokenized'] = R_frm['리뷰'].apply(lambda x: okt.pos(x, norm=True, stem=True))
        R_frm['tokenized'] = R_frm['tokenized'].apply(
            lambda x: [word for word, shape in x if shape in ['Verb', 'Adjective', 'Noun', 'VerbPrefix'] if word not in stopwords])
        R_pred = R_frm['tokenized'].values
        return R_pred

    def model_test(self):
        tokenizer = self.tokenizer
        X_test = tokenizer.texts_to_sequences(self.X_test)
        y_test = self.y_test
        X_test = pad_sequences(X_test, maxlen=self.max_len)
        print("\n 테스트 정확도: %.4f" % (self.loaded_model.evaluate(X_test, y_test)[1]))

    def word_index(self):
        tokenizer = self.tokenizer
        print(tokenizer.word_index)

    def predict(self):
        R_pred = self.preprocessing()
        max_len = self.max_len
        tokenizer = self.tokenizer
        result = []
        pred = tokenizer.texts_to_sequences(R_pred)
        pred = pad_sequences(pred, maxlen=max_len)
        score = self.loaded_model.predict(pred)
        return score

    def prediction(self):
        result = []
        score = self.predict()
        for num in range(len(score)):
            if score[num][0] > 0.9:
                result.append('0')
            elif score[num][0] < 0.1:
                result.append('1')
            else:
                result.append('2')
        return result


In [55]:
# youngho/nare

import pyodbc

DSNNAME = "Tibero6"
DBUSER = "youngho"
DBPWD = "nare"

cnxn = pyodbc.connect('DSN='+DSNNAME+';UID='+DBUSER+';PWD='+DBPWD)
cursor = cnxn.cursor()
cursor.execute("SELECT REST_ID , REVIEW , LABEL_TF from REVIEW_TF;")

# 1개 row만 가져오기
# row = cursor.fetchone()
# 모든 row 가져오기
row2 = cursor.fetchall()
# while row :
#     print(row[0])
#     row = cursor.fetchone()
#     # row = cursor.fetchall()
# print(row)

cnxn.commit()
cnxn.close()

In [40]:
### youngho/nare

import pyodbc

DSNNAME = "Tibero6"
DBUSER = "JH"
DBPWD = "nare"

cnxn = pyodbc.connect('DSN='+DSNNAME+';UID='+DBUSER+';PWD='+DBPWD)
cursor = cnxn.cursor()
cursor.execute("SELECT REST_ID , REVIEW from youngho.REVIEW_TF;")

# 1개 row만 가져오기
# row = cursor.fetchone()
# 모든 row 가져오기
row = cursor.fetchall()
# while row :
#     print(row[0])
#     row = cursor.fetchone()
#     # row = cursor.fetchall()
# print(row)

cnxn.commit()
cnxn.close()

In [41]:
len(row)

77509

In [37]:
type(row[0])

tuple

In [38]:
import pandas as pd

row = [
    ('1713847517', '너무맛있다', 'F'),
    ('1713847517', '최고 제주시내 최고인것같음', 'T'),
    ('1713847517', '분위기도 좋고\n맛도 좋고\n친절한 설명까지~!^^\n행복한 시간이었습니다#\n감사합니다~!', 'T'),
    ('1713847517', '리뷰를 미리 남겨버렸네요 ?\n분위기 좋고 친절한 직원분들덕에 제주도에서 분위기내고 식사할 수 있었어요~~^^', 'F'),
    ('1713847517', '전 양고기스테이크 맛있었고 디저트 코코파인도 맛있었어요!!\n소고기스테이크는 레어로 구워야 덜 질길 것 같아요! 스테이크 다 미디움레어로 했는데 소고기는 좀 많이 익은 느낌이었고 오리는 보통 양고기는 딱 좋았어요!!\n같이 왔던 지인은 오리고기스테이크가 제일 특별하고 맛있었대요!! 어디가서 먹어보기 힘든 음식이라고 맛있었다고 하더라구요~~\n그리고 직원분이 와인 설명도 잘 해주시고 나중에 대리도 불러주시고 엄청 친절하셨어요 감사합니다~\n버섯 리조또(?)가 평이 엄청 좋아서 다음에는 버섯 리조또랑 뇨끼 먹어보고 싶어요 ㅎㅎ\n분위기도 좋고 맛도 있어서 제주도에서 특별한 저녁식사 하고가용 ☺️', 'T')
]

# DataFrame으로 변환
df = pd.DataFrame(row, columns=['REST_ID', 'REVIEW', 'LABEL_TF'])

# 결과 출력
print(df)


      REST_ID                                             REVIEW LABEL_TF
0  1713847517                                              너무맛있다        F
1  1713847517                                     최고 제주시내 최고인것같음        T
2  1713847517  분위기도 좋고\n맛도 좋고\n친절한 설명까지~!^^\n행복한 시간이었습니다#\n감사...        T
3  1713847517  리뷰를 미리 남겨버렸네요 ?\n분위기 좋고 친절한 직원분들덕에 제주도에서 분위기내고...        F
4  1713847517  전 양고기스테이크 맛있었고 디저트 코코파인도 맛있었어요!!\n소고기스테이크는 레어로...        T


In [102]:
data = row
#data

In [103]:
lst_id = []
lst_review = []
for i, r in row :
    lst_id.append(i)
    lst_review.append(r)

In [104]:
row_frm = pd.DataFrame({'REST_ID' : lst_id , 'REVIEW' : lst_review})
row_json = row_frm[['REVIEW']]

In [90]:
row_frm = pd.DataFrame({'REST_ID' : lst_id , 'REVIEW' : lst_review})
row_json = row_frm[['REVIEW']]
json_row = row_json.to_json(orient='records', force_ascii=False)


In [96]:
# json_data = json.loads(json_row)
# for review in json_data :
#     print(review['REVIEW'])

In [ ]:
class psng_predict_all_v2:
    def __init__(self, df):
        self.df = df
        path = ''
        self.loaded_model = load_model(f'{path}posneg_no_oneword.h5')
        train_data = pd.read_csv(f'{path}train_data_main_v2.csv', encoding='euc-kr')
        
        X_train = train_data['tokenized'].apply(eval).values
        vocab_size = 20894 # change
        self.tokenizer = Tokenizer(vocab_size, oov_token='OOV')
        self.tokenizer.fit_on_texts(X_train)
        self.max_len = 250

    def preprocessing(self):
        okt = Okt()
        R_frm = self.df.copy()
        R_frm['리뷰'] = R_frm['리뷰'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
        R_frm['리뷰'].replace('', np.nan, inplace=True)
        R_frm['리뷰'] = R_frm['리뷰'].astype(str)
        stopwords = ['도', '는', '다', '의', '가', '이', '은',
                     '한', '에', '하', '고', '을', '를', '인', '듯',
                     '과', '와', '네', '들', '듯', '지', '임', '게',
                     '는', '이', '했', '슴', '음', '것', '거', '로',
                     '들', '거', '곳', '분', '원', '더', '왜', '해',
                     '수', '할', '그', '함', '돈', '번', '두', '개',
                     '건', '내', '저', '만', '갈', '걸', '제', '명',
                     '분',
                     '인데', '이가', '했었', '해서',
                     '습니다', '했는데', '입니다']

        okt = Okt()
        # R_frm['tokenized'] = R_frm['리뷰'].apply(okt.pos)
        R_frm['tokenized'] = R_frm['리뷰'].apply(lambda x: okt.pos(x, norm=True, stem=True))
        R_frm['tokenized'] = R_frm['tokenized'].apply(
            lambda x: [word for word, shape in x if shape in ['Verb', 'Adjective', 'Noun', 'VerbPrefix'] if word not in stopwords])
        R_pred = R_frm['tokenized'].values
        return R_pred

    def model_test(self):
        tokenizer = self.tokenizer
        X_test = tokenizer.texts_to_sequences(self.X_test)
        y_test = self.y_test
        X_test = pad_sequences(X_test, maxlen=self.max_len)
        print("\n 테스트 정확도: %.4f" % (self.loaded_model.evaluate(X_test, y_test)[1]))

    def word_index(self):
        tokenizer = self.tokenizer
        print(tokenizer.word_index)

    def predict(self):
        R_pred = self.preprocessing()
        max_len = self.max_len
        tokenizer = self.tokenizer
        result = []
        pred = tokenizer.texts_to_sequences(R_pred)
        pred = pad_sequences(pred, maxlen=max_len)
        score = self.loaded_model.predict(pred)
        return score

    def prediction(self):
        result = []
        score = self.predict()
        for num in range(len(score)):
            if score[num][0] > 0.9:
                result.append('0')
            elif score[num][0] < 0.1:
                result.append('1')
            else:
                result.append('2')
        return result
